In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras.utils import to_categorical

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 关闭 TensorFlow 警告日志

In [2]:
# 加载数据
dataset = np.load(r"C:\Users\Maomao Gao\Desktop\5328ass2\datasets\FashionMNIST0.3.npz")
Xtr, Str = dataset["Xtr"], dataset["Str"]
Xts, Yts = dataset["Xts"], dataset["Yts"]

print(Xtr.shape, Str.shape, Xts.shape, Yts.shape)

(18000, 784) (18000,) (3000, 784) (3000,)


In [3]:
# transition matrix (已知)
T = np.array([
    [0.7, 0.3, 0.0],
    [0.0, 0.7, 0.3],
    [0.3, 0.0, 0.7]
])

In [4]:
def training_model(loss, input_shape, num_classes, T):
    model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=X_train.shape[1:]),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])
    model.compile(
        optimizer='adam',
        loss=loss,
        metrics=['accuracy']
    )
    return model

## Forward Learning

In [6]:
def forward_correction_loss(T):
    T = tf.constant(T, dtype=tf.float32)
    def loss(y_true, y_pred):
        y_pred_corrected = tf.matmul(y_pred, T)
        y_pred_corrected = tf.clip_by_value(y_pred_corrected, 1e-7, 1.0)
        return tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true, y_pred_corrected))
    return loss

In [7]:
# 记录多次结果
# num_runs = 10
num_runs = 1 # for quickly testing the implement.
test_accuracies = []

for run in range(num_runs):
    print(f"\n🔁 Run {run+1}/{num_runs}")

    # 每次随机划分数据
    X_train, X_val, S_train, S_val = train_test_split(
        Xtr, Str, test_size=0.2, stratify=Str, random_state=run
    )

    # 归一化
    X_train = X_train.astype("float32") / 255.0
    X_val = X_val.astype("float32") / 255.0
    X_test = Xts.astype("float32") / 255.0

    # One-hot 编码
    S_train_oh = to_categorical(S_train, num_classes=3)
    S_val_oh   = to_categorical(S_val, num_classes=3)   # ✅ 验证集
    Y_test_oh  = to_categorical(Yts, num_classes=3)

    # 创建 Forward Learning模型
    model = training_model(loss=forward_correction_loss(T), input_shape=X_train.shape[1:], num_classes=3, T=T)

    # 训练
    model.fit(
    X_train, S_train_oh,
    validation_data=(X_val, S_val_oh),  # ← 用于早停或监控验证集
    epochs=10,
    batch_size=128,
    verbose=2
    )

    # 在测试集上评估
    loss, acc = model.evaluate(X_test, Y_test_oh, verbose=0)
    test_accuracies.append(acc)
    print(f"✅ Test accuracy = {acc:.4f}")

# 计算平均值和标准差
mean_acc = np.mean(test_accuracies)
std_acc = np.std(test_accuracies)

print("\n📊 Final Performance (10 runs):")
print(f"Mean Test Accuracy: {mean_acc:.4f}")
print(f"Standard Deviation: {std_acc:.4f}")


🔁 Run 1/1
Epoch 1/10
113/113 - 4s - loss: 0.6864 - accuracy: 0.6756 - val_loss: 0.6528 - val_accuracy: 0.6861 - 4s/epoch - 39ms/step
Epoch 2/10
113/113 - 3s - loss: 0.6537 - accuracy: 0.6841 - val_loss: 0.6492 - val_accuracy: 0.6892 - 3s/epoch - 23ms/step
Epoch 3/10
113/113 - 3s - loss: 0.6470 - accuracy: 0.6865 - val_loss: 0.6449 - val_accuracy: 0.6911 - 3s/epoch - 25ms/step
Epoch 4/10
113/113 - 3s - loss: 0.6379 - accuracy: 0.6892 - val_loss: 0.6378 - val_accuracy: 0.6919 - 3s/epoch - 23ms/step
Epoch 5/10
113/113 - 2s - loss: 0.6348 - accuracy: 0.6906 - val_loss: 0.6429 - val_accuracy: 0.6914 - 2s/epoch - 22ms/step
Epoch 6/10
113/113 - 3s - loss: 0.6294 - accuracy: 0.6919 - val_loss: 0.6354 - val_accuracy: 0.6922 - 3s/epoch - 22ms/step
Epoch 7/10
113/113 - 3s - loss: 0.6309 - accuracy: 0.6922 - val_loss: 0.6368 - val_accuracy: 0.6900 - 3s/epoch - 23ms/step
Epoch 8/10
113/113 - 2s - loss: 0.6258 - accuracy: 0.6934 - val_loss: 0.6406 - val_accuracy: 0.6892 - 2s/epoch - 22ms/step
Epoch

## Backward Learning

In [9]:
print(tf.linalg.pinv(T))

tf.Tensor(
[[ 1.32432432 -0.56756757  0.24324324]
 [ 0.24324324  1.32432432 -0.56756757]
 [-0.56756757  0.24324324  1.32432432]], shape=(3, 3), dtype=float64)


In [10]:
print(tf.linalg.inv(T))

tf.Tensor(
[[ 1.32432432 -0.56756757  0.24324324]
 [ 0.24324324  1.32432432 -0.56756757]
 [-0.56756757  0.24324324  1.32432432]], shape=(3, 3), dtype=float64)


In [11]:
def backward_correction_loss(T):
    T = tf.constant(T, dtype=tf.float32)
    # T_inv = tf.linalg.inv(T)
    T_inv = tf.linalg.pinv(T)
    # 用伪逆矩阵计算替换逆矩阵计算，以使结果更加稳定，缓解逆矩阵计算过程中带来的真实标签预测向量不是概率的问题（虽然加和为1，但是有负的，还有大于1的）
    # 但本例子求出来两个计算结果一样。这个问题也仍然存在。
    def loss(y_true, y_pred):
        y_true_corrected = tf.matmul(y_true, T_inv)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0)
        return tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true_corrected, y_pred))
    return loss

In [30]:
# 记录多次结果
# num_runs = 10
num_runs = 1 # for quickly testing the implement.
test_accuracies = []

for run in range(num_runs):
    print(f"\n🔁 Run {run+1}/{num_runs}")

    # 每次随机划分数据
    X_train, X_val, S_train, S_val = train_test_split(
        Xtr, Str, test_size=0.2, stratify=Str, random_state=run
    )

    # 归一化
    X_train = X_train.astype("float32") / 255.0
    X_val = X_val.astype("float32") / 255.0
    X_test = Xts.astype("float32") / 255.0

    # One-hot 编码
    S_train_oh = to_categorical(S_train, num_classes=3)
    S_val_oh   = to_categorical(S_val, num_classes=3)   # ✅ 验证集
    Y_test_oh  = to_categorical(Yts, num_classes=3)

    # 创建 Backward Learning模型
    model = training_model(loss=backward_correction_loss(T), input_shape=X_train.shape[1:], num_classes=3, T=T)

    # 训练
    model.fit(
    X_train, S_train_oh,
    validation_data=(X_val, S_val_oh),  # ← 用于早停或监控验证集
    epochs=10,
    batch_size=128,
    verbose=2
    )

    # 原来忘了输出结果左乘T逆矩阵了 - 在测试集上评估
    # 结果很好，因为此时模型已经学到了真实的标签分布
    loss, acc = model.evaluate(X_test, Y_test_oh, verbose=0)
    test_accuracies.append(acc)
    print(f"✅ Test accuracy (wrong) = {acc:.4f}")

    # 现在补上了 - 在测试集上评估 - 原作者的实现也没有在这里修正。
    # 结果反而差了，可能乘上之后对结果修正过头了
    y_pred_noisy = model.predict(X_test)
    T_inv = np.linalg.pinv(T)
    y_pred_clean = np.dot(y_pred_noisy, T_inv)
    y_pred_clean = np.clip(y_pred_clean, 1e-7, 1.0)
    y_pred_clean /= np.sum(y_pred_clean, axis=1, keepdims=True)
    
    y_pred_label = np.argmax(y_pred_clean, axis=1)
    true_label = np.argmax(Y_test_oh, axis=1)
    acc_clean = np.mean(y_pred_label == true_label)
    print(f"✅ Corrected Test Accuracy (using T⁻¹, correct): {acc_clean:.4f}")
    test_accuracies.append(acc_clean)

# 计算平均值和标准差
mean_acc = np.mean(test_accuracies)
std_acc = np.std(test_accuracies)

print("\n📊 Final Performance (10 runs):")
print(f"Mean Test Accuracy: {mean_acc:.4f}")
print(f"Standard Deviation: {std_acc:.4f}")


🔁 Run 1/1
Epoch 1/10
113/113 - 3s - loss: 0.2009 - accuracy: 0.6697 - val_loss: -6.7740e-03 - val_accuracy: 0.6831 - 3s/epoch - 30ms/step
Epoch 2/10
113/113 - 2s - loss: 0.0982 - accuracy: 0.6803 - val_loss: -2.4639e-03 - val_accuracy: 0.6825 - 2s/epoch - 20ms/step
Epoch 3/10
113/113 - 2s - loss: 0.0986 - accuracy: 0.6782 - val_loss: 0.0086 - val_accuracy: 0.6872 - 2s/epoch - 19ms/step
Epoch 4/10
113/113 - 2s - loss: 0.0125 - accuracy: 0.6837 - val_loss: -7.2837e-03 - val_accuracy: 0.6850 - 2s/epoch - 19ms/step
Epoch 5/10
113/113 - 2s - loss: -3.4413e-02 - accuracy: 0.6820 - val_loss: 1.1399e-04 - val_accuracy: 0.6864 - 2s/epoch - 20ms/step
Epoch 6/10
113/113 - 2s - loss: -1.2384e-02 - accuracy: 0.6776 - val_loss: 0.0210 - val_accuracy: 0.6792 - 2s/epoch - 19ms/step
Epoch 7/10
113/113 - 2s - loss: -9.2740e-02 - accuracy: 0.6818 - val_loss: -1.3190e-02 - val_accuracy: 0.6847 - 2s/epoch - 19ms/step
Epoch 8/10
113/113 - 2s - loss: -9.1506e-02 - accuracy: 0.6822 - val_loss: 0.1301 - val_a

训练过程中loss不稳定，说明这个方法鲁棒性弱于FL。